In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

from sklearn.model_selection import train_test_split
from keras.models import load_model

import shap

c:\Users\Phine\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x = pd.read_csv("../../../Datos/Procesados/x_values.csv", dtype = int)
y_cdm = pd.read_csv("../../../Datos/Procesados/y_dummies_cdm.csv", dtype = int)
y_grdt = pd.read_csv("../../../Datos/Procesados/y_dummies_grdtipo.csv", dtype = int)
y_grd = pd.read_csv("../../../Datos/Procesados/y_dummies_grd.csv", dtype = int)
y_sev = pd.read_csv("../../../Datos/Procesados/y_dummies_sev.csv", dtype = int)

In [3]:
master = 3
y = [y_cdm, y_grd, y_grdt, y_sev]
y_str = ['CDM', 'GRD', 'GRDT', 'SEV']
path_ = f"SHAP Datos/{y_str[master]}/shap_values_{y_str[master]}_datasize_0.0075.csv"

In [4]:
# Leer el archivo CSV y convertirlo en un DataFrame de pandas
shap_values_read = pd.read_csv(path_)

# Leer las dimensiones originales desde el archivo de texto
with open(f"SHAP Datos/{y_str[master]}/shap_values_{y_str[master]}_dimensions.txt", "r") as f:
    dimensions = tuple(map(int, f.read().split(",")))

# Convertir el DataFrame de pandas en una matriz numpy y remodelar a su forma original de 3 dimensiones
shap_values = shap_values_read.values.reshape(dimensions)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y[master], train_size= 0.0075, test_size = 0.0075, random_state = 0, stratify = y[master])

In [6]:
shap_values_converted = [shap_values[i] for i in range(shap_values.shape[0])]

In [10]:
type_plot = 'BAR PLOT'

# Establecer la cantidad de gráficos y el número de filas aleatorias
n_plots = 500
n_random_rows = 500

# Establecer la semilla para garantizar resultados consistentes
np.random.seed(42)

for i, class_shap_values in enumerate(shap_values):
    
    if i not in  [8]:
        continue

    # Generar índices aleatorios
    random_indices = np.random.choice(shap_values_converted[i].shape[0], n_random_rows, replace=False)
    for j in range(n_plots):
        # Seleccionar una fila aleatoria de los valores SHAP
        random_shap_values = class_shap_values[random_indices[j], :]

        # Crear una figura para cada clase
        plt.figure(figsize = [18, 18], dpi = 160)

        # Generar el bar_plot para la clase actual y el valor SHAP seleccionado
        shap.bar_plot(
            random_shap_values,
            feature_names=x_train.columns,
            max_display=25,
            show=False  # Establecer en False para evitar que se muestre la trama en la ventana de salida
        )

        # Guardar cada gráfico en un archivo separado
        output_folder = f"../../../Visualización de datos/SHAP/{y_str[master]}/{type_plot}/{y_train.columns[i]}"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        plt.savefig(f"{output_folder}/bar_plot_value_{j}.png")

        # Cerrar la figura para liberar memoria
        plt.close()

In [8]:
# type_plot = 'SUMMARY PLOT'

# # Iterar sobre la lista de shap_values (cada elemento corresponde a una clase)
# for i, class_shap_values in enumerate(shap_values_converted):
#     # Crear una figura para cada clase
#     plt.figure()

#     # Generar el summary_plot para la clase actual
#     fig = shap.summary_plot(
#         class_shap_values,
#         x_test,
#         feature_names=x_train.columns,
#         class_names=y_train.columns,
#         plot_size=(32, 32),
#         max_display=68,
#         title=f"Clase {i}: {y_train.columns[i]}",
#         show = False
#     )

#     # Guardar cada gráfico en un archivo separado
#     output_folder = f"../../../Visualización de datos/SHAP/{y_str[master]}/{type_plot}"
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
#     plt.savefig(f"{output_folder}/summary_plot_class_{y_train.columns[i]}.png")

#     # Cerrar la figura para liberar memoria
#     plt.close()


In [9]:
# shap.summary_plot(
#     shap_values_converted, 
#     x_test, 
#     feature_names=x_test.columns, 
#     class_names = y_train.columns, 
#     plot_size = (32, 32), 
#     max_display = 68)